In [3]:
import pandas as pd

In [1]:
def concat_dfs(folder_path):
    # folder_path = 'parsed_data/auxilio_emergencial'

    file_names = os.listdir(folder_path)

    df = pd.DataFrame([])

    total_rows = 0

    for file in file_names:
        date = file[:-4]
        path = os.path.join(folder_path, file)
        if df.empty:
            df = pd.read_csv(path)
            df['date'] = pd.to_datetime(date, format='%Y%m')
            total_rows += len(df)
        else:
            next_df = pd.read_csv(path)
            next_df['date'] = pd.to_datetime(date, format='%Y%m')
            total_rows += len(next_df)
            df = pd.concat([df, next_df], ignore_index=True)
    
    df.sort_values('date', ascending=True)

    assert len(df) == total_rows
            
    return df

In [7]:
auxilio_emergencial = concat_dfs('parsed_data/auxilio_emergencial')
bolsa_familia = concat_dfs('parsed_data/bolsa_familia')

In [15]:
bolsa_familia[(bolsa_familia['date'] >= '2021-01-01') & (bolsa_familia['date'] <= '2021-12-31')].groupby('date').sum()

,municipio_siafi,contagem,soma
date,,,
2021-01-01,25175722,14060674.0,2.688952e+09
2021-02-01,25175722,14063377.0,2.638217e+09
2021-03-01,25175722,14381754.0,2.689139e+09
2021-04-01,25175722,4046920.0,1.033779e+09
2021-05-01,25175722,4307712.0,1.103672e+09
2021-06-01,25175722,4299518.0,1.100846e+09
2021-07-01,25175722,4519854.0,1.147867e+09
2021-08-01,25175722,4731487.0,1.154335e+09
2021-09-01,25175722,4664558.0,1.172707e+09
